In [ ]:
spark.stop()

In [4]:
import time
toc =time.time()
import os
import re
import pyspark
import pandas as pd
from functools import reduce
from pyspark.sql import SparkSession
from pyspark.sql.functions import max as _max, min as _min,round
from pyspark.sql.functions import *
from pyspark.sql.functions import col, sqrt,sum,avg,count,explode
from operator import add
from pyspark.sql.types import IntegerType
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.sql.functions import trim
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql.functions import *

spark.stop()

print("Initiating Spark Cluster")
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '20g'), 
                                   ('spark.executor.instances', '12'),
                                   ('spark.driver.cores','1'), 
                                   ('spark.executor.cores', '4'), 
                                   ('spark.sql.broadcastTimeout', '36000'),
                                   ('spark.yarn.queue', 'root.araadh_q1.uhc_oa_pi_dev_sq1'),
                                   ('spark.driver.memory','5g')])
spark = SparkSession.builder.appName("Model-C").config(conf=conf).enableHiveSupport().getOrCreate().newSession()

##############################################################################################################################

#Lead Repository File
lead_gen = spark.read.parquet("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HNP_HSD/Final_HNP")
#Select the hnp leads 
lead_gen = lead_gen.filter(col("hsd_flag")=="yes")
lead_gen.createOrReplaceTempView("lead_gen")

#Lead Release File
lead_release = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HSD_Lead_Release_new.csv",header=True)

#Get the priority orders of teams-priority order determines to which team leads should be shared first
lead_release = lead_release.withColumn("Priority_Order",lead_release["Priority_Order"].cast(IntegerType()))
po = lead_release.select("Priority_Order")
priority_order = [int(row.Priority_Order) for row in po.collect()]
priority_order.sort()

#
for po in priority_order:
    lead_gen_new=lead_gen
    
    #Deduping the shared leads
    dedupe_file = spark.read.csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/de-dupe.csv",header=True)
    print("Dedupe_File")
    dedupe_file.show()
    lead_release_new = lead_release.filter(lead_release["priority_order"]==po)
    team = lead_release_new.select("Team").collect()[0][0]
    print("Team: ",team)
    dedupe = lead_release_new.select("Dedupe").collect()[0][0]
    print("Dedupe: ",dedupe)
    if(dedupe=="All"):
        lead_gen_new = lead_gen_new.join(dedupe_file,lead_gen_new.prov_npi==dedupe_file.prov_npi, how="leftanti")
    else:
        dedupe_file = dedupe_file.withColumn("flag",col("Team").rlike(i))
        dedupe_file_sub = dedupe_file.filter(col("flag")==True)
        lead_gen_new = lead_gen_new.join(dedupe_file_sub,lead_gen_new.prov_npi==dedupe_file_sub.prov_npi, how="leftanti")
    lead_gen_new.createOrReplaceTempView("lead_gen_new") 
    
    print("Categorical Filters")
    lead_release_new = lead_release_new.withColumn("catfil_arr",split(lead_release_new["Categorical_Filters"],";"))
    cat_filter = lead_release_new.select("catfil_arr").collect()[0][0]
    cat_filter

    col_name = ""
    col_value = ""
    if not (cat_filter is None):
        for i in cat_filter:
            col_name = i.split(":")[0].strip()
            col_value = i.split(":")[1].strip()
            col_value = col_value[1:len(col_value)-1]
            col_value_list = col_value.split(",")
            lead_gen_new = lead_gen_new.withColumn("cat_flag_"+col_name,concat_ws(",",lead_gen_new[col_name]))
            for j in col_value_list:
                print("j: ",j)
                lead_gen_new = lead_gen_new.withColumn("categorical_flag_"+j, lead_gen_new["cat_flag_"+col_name].rlike(j))
                lead_gen_new = lead_gen_new.withColumn("categorical_flag_"+j, lead_gen_new["categorical_flag_"+j].cast(StringType()))
    else:
        print("Category Filter is Empty")

    print("Numerical Filters")
    lead_release_new = lead_release_new.withColumn("num_fil_arr",split(lead_release_new["Numeric_Filters"],";"))
    num_filter = lead_release_new.select("num_fil_arr").collect()[0][0]
    num_filter

    
    col_name = ""
    col_value = ""
    if not(num_filter is None):
        for i in num_filter:
            i = i.strip()
            col_name = i.split(":")[0].strip()
            col_value = i.split(":")[1].strip()
            col_value = col_value[1:len(col_value)-1]
            col_value_list = col_value.split(",")
            col_str =col_name+"_str"
            print("col_name: ",col_name)
            print("col_value: ",col_value)
            print("col_value_list: ",col_value_list)
            lead_gen_new = lead_gen_new.withColumn(col_str,lead_gen_new[col_name].cast(StringType()))
            for j in col_value_list:
                j_list = re.split('([^a-zA-Z0-9])',j)
                col_name = j_list[0].strip()
                sign = j_list[1].strip()
                num = j_list[2].strip()
                #val = col_name[0:len(col_name)-1]
                #num = j.split(" ")[1].strip()
                #num = float(num)
                reg ="((\d+)|(\d+\.\d+))(?=:"+col_name+")" 
                #print("j_list :",j_list)
                #print("col_name: ",col_name)
                #print("cl:",cl)
                #print("val",val)
                #print("num",num)
                #print("sign",sign)
                #print("num",num)
                #a =cl+"_str"
                lead_gen_new = lead_gen_new.withColumn("num_"+col_name,regexp_extract(lead_gen_new[col_str],reg,1))
                lead_gen_new = lead_gen_new.withColumn("num_"+col_name,lead_gen_new["num_"+col_name].cast('float'))
                lead_gen_new = lead_gen_new.fillna(0)
                if sign==">":
                    lead_gen_new = lead_gen_new.withColumn("numeric_flag_"+col_name, when(lead_gen_new["num_"+col_name] >= num,"true").otherwise("false"))
                else:
                    lead_gen_new = lead_gen_new.withColumn("numeric_flag_"+col_name, when(lead_gen_new["num_"+col_name] <= num,"true").otherwise("false"))
    else:
        print("Numeric Filter is empty")
        

    lead_gen_new_columns = lead_gen_new.columns
    
    #Apply the filters to the lead repository file
    fil_col_list = [x for x in lead_gen_new_columns if ("categorical_flag" in x or "numeric_flag" in x)]
    print("flag_col_list:\n",fil_col_list)
    filter_query=""
    if (len(fil_col_list)!=0):
        for i in fil_col_list:
            lead_gen_new = lead_gen_new.withColumn(i,when(lead_gen_new[i]=="true","1").otherwise("0"))
            lead_gen_new.createOrReplaceTempView("lead_gen_new")
            filter_query =filter_query+" and "+i+" ==0"
        filter_query=filter_query[5:len(filter_query)]
        print("filter_query: ",filter_query)       
        lead_gen_fil = spark.sql("select a.* from lead_gen_new a where "+filter_query)
    else:
        lead_gen_fil = spark.sql("select * from lead_gen_new")
    print("#Leads after filtering",lead_gen_fil.count())
    #Extracting top CPT,DOS,DIAG etc. from the lead release file
    dos = int(lead_release_new.select("DOS").collect()[0][0])
    diag = int(lead_release_new.select("DIAG").collect()[0][0])
    cpt = int(lead_release_new.select("CPT").collect()[0][0])
    leads= int(lead_release_new.select("Leads").collect()[0][0])
    
    #Sorting the leads
    lead_gen_fil =lead_gen_fil.orderBy(lead_gen_fil["DOS_P12M_Cnt"].desc()).limit(leads)
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
    print("#Leads extracted for",team,lead_gen_fil.count())
    
    array_columns = ['DOS_P12M_PatCnt','CPT_P12M_Cnt','CPT_P9M_Cnt','CPT_P6M_Cnt','CPT_P3M_Cnt','Diag_P12M_Cnt','Diag_P9M_Cnt','Diag_P6M_Cnt','Diag_P3M_Cnt']
    #Extracting top cpt,diag etc. from the array columns
    for i in array_columns:
        print(i)
        temp = lead_gen.select("prov_npi",explode(i).alias("array_val"))
        temp = temp.withColumn("key",trim(split(temp["array_val"],":").getItem(0)))
        temp = temp.withColumn("key",temp["key"].cast("float"))
        window = Window.partitionBy(temp['prov_npi']).orderBy(temp["key"].desc())
        temp1 = temp.select('*', rank().over(window).alias('rank'))
        if "DOS" in i:
            temp1 = temp1.filter(temp1["rank"] <= dos)
        elif "CPT" in i:
            temp1 = temp1.filter(temp1["rank"] <= cpt)
        elif "Diag" in i:
            temp1 = temp1.filter(temp1["rank"] <= diag)
        temp1 = temp1.groupby("prov_npi").agg(collect_set("array_val").alias(i+"_new"))
        temp1.createOrReplaceTempView("temp1")
        lead_gen_fil = spark.sql("select a.*,"+" b."+i+"_new"+" from lead_gen_fil a left join temp1 b on a.prov_npi = b.prov_npi")
        lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
    
    #Fetching the columns required by the team
    new_col = [x for x in lead_gen_fil.columns if "new" in x]
    print("new_col:\n",new_col)
    
    for i in new_col:
        i_sub = i[0:len(i)-4]
        lead_gen_fil = lead_gen_fil.drop(i_sub).withColumnRenamed(i,i_sub)
        lead_gen_fil = lead_gen_fil.withColumn(i_sub,f.col(i_sub).cast(StringType()))
    lead_gen_fil.createOrReplaceTempView("lead_gen_fil")
       
    lead_release = lead_release.withColumn("feat_arr",split(lead_release["Features"],","))
    selected_cols = lead_release.select("feat_arr").collect()[0][0]
    lead_gen_sub = lead_gen_fil.select(selected_cols)
    #lead_gen_sub.cache()
    print("writing file")
    wrt_toc=time.time()
    lead_gen_sub.coalesce(1).write.mode("overwrite").option("header","true").csv("/datalake/uhc/ei/pi_ara/kappa/users/aaiman/HNP_HSD/"+str(team)+"hsd.csv")
    print("Done writing file")
    wrt_tic=time.time()
    #print("Time taken in writing the file is: ",(float(wrt_tic)-float(wrt_toc))/60)
    #lead_gen_sub.unpersist()
    #Removing the leads that are shared with one team
    lead_gen_sub.createOrReplaceTempView("lead_gen_sub")
    lead = lead_gen_sub.select("prov_npi")
    lead.createOrReplaceTempView("lead")
    print("Removing the shared leads from lead gen")
    lead_gen = lead_gen.join(lead,lead_gen.prov_npi==lead.prov_npi,how="leftanti")    
    lead_gen.createOrReplaceTempView("lead_gen")
    #lead_gen.select("prov_npi").show()
    print("######################","End of",po,"iteration","#######################")
tic =time.time()
tic=float(tic)
toc =float(toc)
print("Total time taken:",(tic-toc)/60)

Initiating Spark Cluster
Dedupe_File
+----------+----+
|  prov_npi|Team|
+----------+----+
|1649218173|PLRT|
|1427293281| AVS|
|1275791832|PLRT|
|1285660241| AVS|
+----------+----+

Team:  PLRT
Dedupe:  All
Categorical Filters
j:  PNIC
j:  95964
j:  MEDICA
Numerical Filters
Numeric Filter is empty
flag_col_list:
 ['categorical_flag_PNIC', 'categorical_flag_95964', 'categorical_flag_MEDICA']
filter_query:  categorical_flag_PNIC ==0 and categorical_flag_95964 ==0 and categorical_flag_MEDICA ==0
#Leads after filtering 24
#Leads extracted for PLRT 10
DOS_P12M_PatCnt
CPT_P12M_Cnt
CPT_P9M_Cnt
CPT_P6M_Cnt
CPT_P3M_Cnt
Diag_P12M_Cnt
Diag_P9M_Cnt
Diag_P6M_Cnt
Diag_P3M_Cnt
new_col:
 ['DOS_P12M_PatCnt_new', 'CPT_P12M_Cnt_new', 'CPT_P9M_Cnt_new', 'CPT_P6M_Cnt_new', 'CPT_P3M_Cnt_new', 'Diag_P12M_Cnt_new', 'Diag_P9M_Cnt_new', 'Diag_P6M_Cnt_new', 'Diag_P3M_Cnt_new']
writing file
Done writing file
Removing the shared leads from lead gen
###################### End of 1 iteration #######################
